In [135]:
from dotenv  import load_dotenv

In [136]:
import json
import pandas as pd
import traceback
import os

In [137]:
from langchain.chat_models import ChatOpenAI

In [138]:
load_dotenv()

True

In [139]:

apiKey=os.getenv("OPENAI_API_KEY")
if apiKey is None:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")

In [140]:
chat_client=ChatOpenAI(
    openai_api_key=apiKey,
    model="gpt-3.5-turbo",
    temperature=0.5,
    max_tokens=1000,
    n=1
)

In [141]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [142]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [143]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [144]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
 )

In [145]:
quiz_chain=LLMChain(llm=chat_client, prompt=quiz_generation_prompt, verbose=True,  output_key="quiz")

In [146]:
TEMPLATE2="""
You are an expert in english gramary and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [147]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["quiz","subject"], template=TEMPLATE2)

In [148]:
review_chain=LLMChain(llm=chat_client,prompt=quiz_evaluation_prompt,verbose=True,output_key="review")


In [149]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain],  input_variables=["text","number","subject","tone","response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [150]:
with open("../data.txt", "r") as f:
    TEXT=f.read()

In [151]:
TEXT

'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, 

In [152]:
str_res_json=json.dumps(RESPONSE_JSON, indent=4)

In [153]:
str_res_json

'{\n    "1": {\n        "mcq": "multiple choice question",\n        "options": {\n            "a": "choice here",\n            "b": "choice here",\n            "c": "choice here",\n            "d": "choice here"\n        },\n        "correct": "correct answer"\n    },\n    "2": {\n        "mcq": "multiple choice question",\n        "options": {\n            "a": "choice here",\n            "b": "choice here",\n            "c": "choice here",\n            "d": "choice here"\n        },\n        "correct": "correct answer"\n    },\n    "3": {\n        "mcq": "multiple choice question",\n        "options": {\n            "a": "choice here",\n            "b": "choice here",\n            "c": "choice here",\n            "d": "choice here"\n        },\n        "correct": "correct answer"\n    }\n}'

In [154]:
NUMBER=3
SUBJECT="Biology"
TONE="simple"

In [155]:
# how to use the callback to get the cost of the API call
# get the cost of the API call
# https://platform.openai.com/docs/guides/gpt/costs-and-billing
with get_openai_callback() as cb:
    response=generate_evaluate_chain({"text": TEXT, "number": NUMBER, "subject": SUBJECT, "tone": TONE, "response_json": str_res_json})
    print(response)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

You are an expert in english gramary and writer. Given a Multiple Choice Quiz for Biology students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:

{
    "1": {
        "mcq": "What is the scientific study of life called?",
        "options": {
            "a": "Chemistry",
            "b": "Biology",
            "c": "Physics",
            "d": "Geology"
        },
        "correct": "b"
    },
    "2": {
        "mcq": "Which theme in biology explains the unity and diversity of life?",
        "options": {
            "a": "Evolution",
            "b": "Energy processing",
            "c": "Cell structure",
            "d": "Regu

In [156]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1219
Prompt Tokens:961
Completion Tokens:258
Total Cost:0.0019575


In [157]:
quiz=json.loads(response.get("quiz"))

In [158]:
quiz

{'1': {'mcq': 'What is the scientific study of life called?',
  'options': {'a': 'Chemistry',
   'b': 'Biology',
   'c': 'Physics',
   'd': 'Geology'},
  'correct': 'b'},
 '2': {'mcq': 'Which theme in biology explains the unity and diversity of life?',
  'options': {'a': 'Evolution',
   'b': 'Energy processing',
   'c': 'Cell structure',
   'd': 'Regulation of internal environments'},
  'correct': 'a'},
 '3': {'mcq': 'What is the main unit of organization in biology?',
  'options': {'a': 'Atoms', 'b': 'Cells', 'c': 'Molecules', 'd': 'Organs'},
  'correct': 'b'}}

In [159]:
quiz_table_data=[ {
    "question":value["mcq"],
    "options":" | ".join([ f"{option_key}: {option_value}" for option_key,option_value in value["options"].items() ]),
    "correct":value["correct"],

}   for key,value in quiz.items()
 ]

In [160]:
quiz_table_data

[{'question': 'What is the scientific study of life called?',
  'options': 'a: Chemistry | b: Biology | c: Physics | d: Geology',
  'correct': 'b'},
 {'question': 'Which theme in biology explains the unity and diversity of life?',
  'options': 'a: Evolution | b: Energy processing | c: Cell structure | d: Regulation of internal environments',
  'correct': 'a'},
 {'question': 'What is the main unit of organization in biology?',
  'options': 'a: Atoms | b: Cells | c: Molecules | d: Organs',
  'correct': 'b'}]

In [161]:
quiz=pd.DataFrame(quiz_table_data)

In [162]:
quiz.to_csv("./quiz.csv", index=False)

In [1]:
import os
import logging
from datetime import datetime

LOG_FILE=f"{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"


In [2]:
LOG_FILE

'04_17_2025_12_57_27.log'